In [1]:
import os
from dotenv import load_dotenv
import heapq
import pymysql
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from nltk import  word_tokenize
import string
import pandas as pd
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tag import pos_tag


port = PorterStemmer()
wnl = WordNetLemmatizer()


In [2]:
load_dotenv(dotenv_path='../')

db_host = os.getenv('DB_HOST')
db_port = os.getenv('DB_PORT')
db_username = os.getenv('DB_USERNAME')
db_password = os.getenv('DB_PASSWORD')
db_database = os.getenv('DB_DATABASE')

In [3]:
conn = pymysql.connect(host=db_host, port=int(db_port), user=db_username, passwd=db_password, db=db_database)
# cur = conn.cursor()
# cur.execute("SELECT * FROM job_skills")
# print(cur.description)
# for row in cur:
#     print(row[2])
# cur.close()
# conn.close()

In [4]:
skills_dict = {}
cur = conn.cursor()
cur.execute('SELECT c.id, skill_name from companies as c JOIN company_jobs cj ON c.id = cj.company_id JOIN job_skills js ON js.job_id = cj.id')

for row in cur:
    if skills_dict.get(row[0],'') in ('', None):
        skills_dict[row[0]] = ''.join(row[1].lower())
    else:
        final_string = skills_dict.get(row[0]) + ' ' + row[1].lower()
        skills_dict[row[0]] = final_string

cur.close()
print(skills_dict)

{'00a5680b-6143-41e5-bddf-80492efdb823': 'problem solving product knowledge cash wrap inventory management visual updates communication relationship building customer service education phone skills systems usage sales promotions conflict resolution alterations', '00ba046f-f336-4e68-853f-f617304ffc9d': 'stress management assessment infection control cpr blood transfusions critical thinking safety decisionmaking hipaa medication administration documentation implementation leadership acls planning nursing nutrition cultural sensitivity conflict resolution patient care vital signs problemsolving wound care communication teamwork time management iv therapy clinical excellence evaluation emr/ehr osha pain management hygiene medical terminology oxygen therapy collaboration pals', '01a53c5d-8d45-462a-8c59-64aff4249174': 'lpn handson management clinical background managerial skills rn organizational skills medical assistant medical office experience people skills', '03c54baf-442c-410a-a9af-f8f9

In [14]:
user_skills_merged = ''

user_id = '95b068aa-4320-38cf-bdfe-ed189b9d72fa'

cur = conn.cursor()
cur.execute('SELECT skill_name FROM `user_skills` WHERE user_id = \''+ user_id+'\'')
for row in cur:
    user_skills_merged = user_skills_merged + ' '+  row[0].lower()
cur.close()

print(user_skills_merged)

 python sql


In [24]:
def get_label(tag):
    if tag == 'jj':
        return 'a'
    elif tag in ['vb','nn','rb']:
        return tag[0]
    else:
        return None

def lemma(word_list):
    lem = []
    tags = pos_tag(word_list)
    for word, tag in tags:
        label = get_label(tag.lower())
        if(label!=None):
            lem.append(wnl.lemmatize(word,label))
        else:
            lem.append(wnl.lemmatize(word))

    return lem

def preProcess(text):
    tokenized = word_tokenize(text)
    tokenized = [port.stem(word) for word in tokenized if word.isalpha()]
    tokenized = lemma(tokenized)
    return tokenized

vectorizer = CountVectorizer(tokenizer=preProcess,stop_words='english', binary=True) 
sparse_matrix = vectorizer.fit_transform(list(skills_dict.values()))

df = pd.DataFrame(
   sparse_matrix.todense(),
   columns=vectorizer.get_feature_names_out(),
   index=skills_dict.keys(),
)

sparse_matrix_user = vectorizer.transform([user_skills_merged])
df_user = pd.DataFrame(
    sparse_matrix_user.todense(),
    columns=vectorizer.get_feature_names_out(),
    index = [user_id]
)

display(df.head(5))
display(df_user.head(5))

c:\Users\retti\anaconda3\envs\NLP\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\retti\anaconda3\envs\NLP\lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'el', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'far', 'fifti', 'formerli', 'forti', 'ha', 'henc', 'hereaft', 'herebi', 'hi', 'howev', 'hundr', 'inde', 'latterli', 'le', 'mani', 'meanwhil', 'moreov', 'mostli', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'plea', 'seriou', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv'

,abil,abov,absenc,aca,academi,acca,accept,access,accid,accommod,...,xray,year,yield,youth,youtub,zemax,zone,à,écout,écrite
00a5680b-6143-41e5-bddf-80492efdb823,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
00ba046f-f336-4e68-853f-f617304ffc9d,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
01a53c5d-8d45-462a-8c59-64aff4249174,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
03c54baf-442c-410a-a9af-f8f94d186b74,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
07860414-b08c-4ec8-a2e4-42ad2a9568e3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


,abil,abov,absenc,aca,academi,acca,accept,access,accid,accommod,...,xray,year,yield,youth,youtub,zemax,zone,à,écout,écrite
95b068aa-4320-38cf-bdfe-ed189b9d72fa,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
class MaxHeapObj(object):
  def __init__(self, val): 
     self.val = val
  def __lt__(self, other): 
     col,cos = self.val
     col1,cos1 = other.val
     return cos > cos1
  def __eq__(self, other): 
     col,cos = self.val
     col1,cos1 = other.val
     return cos == cos1

cosine_result = cosine_similarity(df,df_user)
heap_result = []
for col,cos in zip(df.index,cosine_result):
    data = (col,cos)
    heapq.heappush(heap_result, MaxHeapObj(data))


result_arr =[]
for i in range(5):
    res = heapq.heappop(heap_result).val
    print(res)
    result_arr.append(res[0])
print(result_arr)

('beb77618-d977-4ad8-b7d0-827ae36e1a40', array([0.24253563]))
('e30359d2-0801-4dc9-bcbd-a9093551a506', array([0.19611614]))
('976149cf-5a65-4922-971f-3a67aae41e4c', array([0.1767767]))
('b813c5db-8ef6-4873-b3ab-1243583aa65a', array([0.17149859]))
('66c12f4e-4e67-4edc-8e8e-7d45f2b60cf6', array([0.12126781]))
['beb77618-d977-4ad8-b7d0-827ae36e1a40', 'e30359d2-0801-4dc9-bcbd-a9093551a506', '976149cf-5a65-4922-971f-3a67aae41e4c', 'b813c5db-8ef6-4873-b3ab-1243583aa65a', '66c12f4e-4e67-4edc-8e8e-7d45f2b60cf6']
